# Homework 3 - What is the best anime in the world?

## 1. Data collection

In [1]:
import requests as r
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
from datetime import datetime
import csv
import re
import nltk

#### List of paths

In [2]:
path_list_anime = "list_anime.txt"
url_of_single_page = "topanime.php?limit="
path_page_general = "Page"
path_documents_file = "documents.tsv"
path_vocabulary_file = "vocabulary.json"
path_index_file = "Uindex.json"

### 1.1. Get the list of animes

#### Write file with all anime urls

In [3]:
# 19130 anime in total, 383 pages with 50 animes

def write_list_anime(path_list_anime, url_of_single_page):
    # Number of total pages
    n_pages = 383 

    with open(path_list_anime, "w", encoding='utf-8') as file:

        for page in tqdm(range(0, n_pages)):

            # URL of the single page 
            url = url_of_single_page + str(page * 50)

            # Get response object
            response = r.get(url)

            # Get html
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all links iterating through the document
            for tag in soup.find_all("tr", class_="ranking-list"):
                a_list = tag.find_all('a', class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True)
                for a in a_list:
                    link = a['href']
                    file.write(str(link) + '\n')

    file.close()

### 1.2. Crawl animes

In [4]:
def download_html(path_list_anime, path_page_general):
    file = open(path_list_anime, "r", encoding='utf-8')

    # Initialize counters 
    counter_anime = 0
    page = 0

    # Get html file for every anime (url is a line in the file list_anime) 
    for line in tqdm(file):
        counter_anime += 1
        if (counter_anime%50 == 1):
            page +=1
        response = r.get(line)
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            os.mkdir(path_page_general + str(page))
        except OSError:
            pass

        with open(path_page_general + str(page) + "/article_" + str(counter_anime) + ".html", "w", encoding='utf-8') as file:
            file.write(str(soup))

### 1.3 Parse downloaded pages

#### Function to retrieve all the information from html

In [5]:
# Retrive information at page i of anime j
def retrive_information(i, j):
        with open(path_page_general + str(i) + "/anime" + str(j) + ".html", "r", encoding='utf-8') as file:
    
            # Read file and parse html code
            content_page = file.read()
            soup = BeautifulSoup(content_page, 'html.parser')
            
            # Retrive information 
            animeTitle = soup.find_all("h1", class_="title-name h1_bold_none")[0].find("strong").text
        
            animeType = soup.find_all('a', href = re.compile(r'type*'))[0].text.strip()
            
            animeNumEpisode = soup.find(string='Episodes:').next_element.replace("\n","").strip()
            
            animeNumMembers = soup.find(string='Members:').next_element.replace("\n","").replace(',','').replace('#', '').strip()
        
            animeScore = soup.find('span', {"itemprop":"ratingValue"}).text.strip()
            
            animeUsers = soup.find('span', {"itemprop":"ratingCount"}).text.strip()
            
            animeRank = soup.find(string='Ranked:').next_element.replace("\n","").replace('#','').strip()
            
            animePopularity = soup.find(string='Popularity:').next_element.replace("\n","").replace('#', '').strip()
            
            animeDescription = soup.find('p',{'itemprop':'description'}).text.replace('\n', '').replace('  ', '').strip()

    
            # Iterate in di with dates   
            dates_div = soup.find_all("div", class_="spaceit_pad")
            for i in range(0, len(dates_div)):
        
                # Dates
                if dates_div[i].find("span", string='Aired:') != None:
                    dates = dates_div[i].contents[2].strip().split("to")
                    try:
                        releaseDate = datetime.strptime(dates[0].strip(), '%b %d, %Y').date()
                    except:
                        releaseDate = None
                    try:
                        endDate = datetime.strptime(dates[1].strip(), '%b %d, %Y').date()
                    except:  
                        endDate = None
    
            # Anime Description
            animeDescription = soup.find("p", itemprop='description').text
    
            # Related Animes
            try:
                related_anime_table = soup.find("table", class_="anime_detail_related_anime")
                links_list = [link.text for link in related_anime_table.find_all("a")]
                animeRelated = list(set(links_list))
            except:
                animeRelated = None
    
            # Characters
            animeCharacters = soup.find_all("h3", class_="h3_characters_voice_actors")
            animeCharacters = [character.text for character in animeCharacters]
    
            # Voices
            animeVoices = soup.find_all("td", class_="va-t ar pl4 pr4")
            animeVoices = [actor.find("a").text for actor in animeVoices]
    
            a_no = []
            total = []
            result = []
            animeStaff = []
    
            # Staff 
            if animeCharacters == [] and animeVoices == []:
                div = soup.find_all("div", class_="detail-characters-list clearfix")[0]
                tds = div.find_all("td", class_=lambda x: x != 'ac borderClass' and x != 'ac' and x == 'borderClass')
    
                smalls = div.find_all("small")
    
                for td in tds:
                    a_no = set(list(td.find_all("a", class_="fw-n")))
                    total = set(list(td.find_all("a")))
                    if list(total.difference(a_no)) != []:
                        result.append(list(total.difference(a_no)))
    
                for (s, role) in zip(result, smalls):
                    s = str(s).strip("[").strip("]")
                    htmlTag = BeautifulSoup(s, "html.parser")
                    animeStaff.append([htmlTag.text, role.text])
                
            # There are no staff
            elif len(soup.find_all("div", class_="detail-characters-list clearfix")) < 2:
                animeStaff = []
                
            # There are one of animeCharacters or animeVoices
            else:
                div = soup.find_all("div", class_="detail-characters-list clearfix")[1]
                
                tds = div.find_all("td", class_=lambda x: x != 'ac borderClass' and x != 'ac' and x == 'borderClass')

                smalls = div.find_all("small")

                for td in tds:
                    a_no = set(list(td.find_all("a", class_="fw-n")))
                    total = set(list(td.find_all("a")))
                    if list(total.difference(a_no)) != []:
                        result.append(list(total.difference(a_no)))

                for (s, role) in zip(result, smalls):
                    s = str(s).strip("[").strip("]")
                    htmlTag = BeautifulSoup(s, "html.parser")
                    animeStaff.append([htmlTag.text, role.text])
            
        # Create list with overall information
        final_information = [animeTitle, animeType,animeNumEpisode,releaseDate,endDate,
                         animeNumMembers, animeScore,animeUsers,animeRank,animePopularity,
                         animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff]
        return final_information

#### Create a .tsv file for each anime and a .tsv file with 3 most important information for all: Title, Description, Url 

In [6]:
def create_tsvs(path_list_anime, path_page_general, path_documents_file):
    # Create .tsv file
    n_pages = 384

    # Collecting all Urls
    with open(path_list_anime, 'r', encoding='utf-8') as f_urls:
        urls = f_urls.readlines()
    f_urls.close

    # Create .tsv with only 3 important fields
    with open(path_documents_file, 'w', encoding='utf-8') as f:
        tsv_writer = csv.writer(f, delimiter='\t')

        # Write haeders
        tsv_writer.writerow(['animeTitle','animeDescription','Url'])

    f.close

    # Iterating through pages and then animes
    for i in range(1, n_pages):

        # Start and end computed with respect to the 
        start_iterate_anime = (i-1)*50+1
        end_iterate_anime = ((i-1)*50+1)+50
        for j in range(start_iterate_anime, end_iterate_anime):

            # Write for each anime a .tsv file
            with open(path_page_general + str(i) + "/anime_" + str(j)+ ".tsv", 'w', encoding='utf-8') as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')

                # Write haeders
                tsv_writer.writerow(['animeTitle', 'animeType','animeNumEpisode','releaseDate','endDate',
                                     'animeNumMembers', 'animeScore','animeUsers','animeRank','animePopularity',
                                     'animeDescription','animeRelated','animeCharacters','animeVoices','animeStaff'])

                # Write information
                information = retrive_information(i, j) 
                tsv_writer.writerow(information)
            out_file.close

            important_information = [information[0],information[10], urls[j]]

            # Append data to .tsv with only 3 important fields
            with open(path_documents_file, 'a', encoding='utf-8') as f:
                tsv_writer = csv.writer(f, delimiter='\t')

                # Write information 
                tsv_writer.writerow(important_information)

            f.close

## 2. Search Engine

### Create pandas dataframe with important information

In [3]:
def df_from_3_information_file(path_documents_file):
    original_df = pd.read_csv(path_documents_file, sep='\t')
    return original_df

In [4]:
original_df = df_from_3_information_file(path_documents_file)

### Text cleaning:

In order we will do the following operations:

- Remove "[Written by MAL Rewrite]" at the end of each description

- Remove Contractions (for example "won't" become "will not" and "don't" become "do not")

- Make all characters lower case (for example "Hello" become "hello")

- Remove dashes

- Remove ordinal numbers (for example 1st, 2nd, ...)

- Remove stopwords (adding to stopwords frequent words that appear in descriptions, for example "character", "end") 

- Remove punctuation

- Stemming

In [5]:
import re
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
import string
import json

nltk.download('stopwords')
default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english')

# Append some recurrent words in synopsis:
frequent_words = ['character','characters','end']
default_stopwords.extend(frequent_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Antonella\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def remove_written_mal_rewrite(text):
    return text.replace('[Written by MAL Rewrite]', '')

In [7]:
def remove_contractions(text):
    # Create patterns
    patterns = [
       (r'won\'t', 'will not'),
       (r'can\'t', 'cannot'),
       (r'i\'m', 'i am'),
       (r'(\w+)\'ll', '\g<1> will'),
       (r'(\w+)n\'t', '\g<1> not'),
       (r'(\w+)\'ve', '\g<1> have'),
       (r'(\w+)\'s', '\g<1> is'),
       (r'(\w+)\'re', '\g<1> are'),
    ]
    
    # Compile patterns with re.compile()
    patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    
    # Substitute words
    for (pattern, repl) in patterns:
        text = re.sub(pattern, repl, text)
    return text

In [8]:
def to_lower_case(text):
    return text.lower()

In [9]:
def replace_dashes(text):
    return text.replace('—',' ')

In [10]:
def remove_ordinal_num(text):
    text = re.sub('[\d]+(st|nd|rd|th)', '', text)
    return text

In [11]:
def tokenize_text(text):
    return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

In [12]:
def remove_punctuation(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(text)
    return ' '.join(new_words)

In [13]:
def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

In [14]:
def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

In [15]:
def clean_text(text):
    
    # Text cleaning
    text = remove_written_mal_rewrite(text)
    text = remove_contractions(text)
    text = to_lower_case(text)
    text = replace_dashes(text)
    text = remove_ordinal_num(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = stem_text(text)
    
    return text

In [16]:
def clean_df(original_df):
    df_descriptions = original_df['animeDescription']
    df_descriptions = pd.DataFrame(df_descriptions)
    df_cleaned = df_descriptions.applymap(lambda x : clean_text(x), na_action='ignore')
    return df_cleaned

In [17]:
df = clean_df(original_df)
df_descriptions = pd.DataFrame(df)

### 2.1. Conjunctive query

### 2.1.1) Create your index!

#### Create vocabulary and save it in a file .json

In [18]:
def write_vocabulary(path_vocabulary_file):    
    # Create a list 
    df_words = df_descriptions.applymap(lambda x : set(tokenize_text(x)), na_action='ignore')
    words_union = set()

    for row in df_words['animeDescription']:
        words_union = words_union.union(set(row))

    words_list = list(words_union)

    # Create vocabulary
    vocabulary = { words_list[i] : i for i in range(len(words_list)) }

    # Write the file
    with open(path_vocabulary_file, 'w', encoding='utf-8') as file:
        json.dump(vocabulary, file)
    file.close

In [19]:
write_vocabulary(path_vocabulary_file)

#### Create index and save it in a file .json

In [20]:
def write_index(path_index_file):
    # Create the index with (id_word : [doc1, doc2, ...])
    with open(path_vocabulary_file, 'r', encoding='utf-8') as f:
        vocabulary = json.load(f)
    df_words = df_descriptions.applymap(lambda x : set(tokenize_text(x)), na_action='ignore')
    index = {}
    for n_row in range(df_words['animeDescription'].shape[0]):
        for (k,v) in vocabulary.items():
            if v not in index.keys():
                index[v] = []
            if k in df_words['animeDescription'][n_row]:
                # n_row+1 becouse n_anime is n_row_in_dataset + 1
                index[v].append(n_row+1)

    # Write the file
    with open(path_index_file, 'w', encoding='utf-8') as file:
        json.dump(index, file)
    file.close

In [21]:
write_index(path_index_file)

### 2.1.2) Execute the query

#### Load vocabulary.json and index.json as python dictionaries

In [22]:
def read_vocabulary_and_index(path_vocabulary_file, path_index_file):
    # Vocabulary
    with open(path_vocabulary_file, 'r', encoding='utf-8') as f:
        vocabulary = json.load(f)
    f.close

    # Index
    with open(path_index_file, 'r', encoding='utf-8') as f:
        index = json.load(f)
    f.close
    
    return vocabulary, index

#### Query

In [23]:
def execute_query(vocabulary, index):
    # Query
    query = input()
    query = clean_text(query).split(" ")

    # Trasform every word of the query in the corrispondant id
    id_words_query = [vocabulary[word] for word in query if word in vocabulary.keys()]

    documents_set = set()
    for id_word in id_words_query:
        if documents_set == set():
            documents_set.update(index[str(id_word)])
        else:
            documents_set = documents_set.intersection(set(index[str(id_word)]))
            if documents_set == set():
                break

    documents = list(documents_set)
    documents_in_dataframe = [x-1 for x in documents]

    # Return rows
    original_df = df_from_3_information_file(path_documents_file)
    return original_df.iloc[documents_in_dataframe]

In [24]:
vocabulary, index = read_vocabulary_and_index(path_vocabulary_file, path_index_file)

In [25]:
vocabulary

{'intern': 0,
 'treat': 1,
 'genuin': 2,
 'keep': 3,
 'pig': 4,
 'spaceship': 5,
 'plug': 6,
 'pet': 7,
 'lack': 8,
 'happen': 9,
 'battleground': 10,
 'wei': 11,
 'death': 12,
 'amanto': 13,
 'hierarchi': 14,
 'yuuji': 15,
 'adversari': 16,
 'idofront': 17,
 'yabuki': 18,
 'parti': 19,
 'time': 20,
 'will': 21,
 'taichi': 22,
 'sukuna': 23,
 'present': 24,
 'peac': 25,
 'sourc': 26,
 'realiti': 27,
 'limb': 28,
 'defin': 29,
 'game': 30,
 'muster': 31,
 'seriou': 32,
 'threaten': 33,
 'clubroom': 34,
 'confid': 35,
 'kaiser': 36,
 'alien': 37,
 'casualti': 38,
 'startl': 39,
 'solitari': 40,
 'doll': 41,
 'studio': 42,
 'ingredi': 43,
 'turn': 44,
 'fix': 45,
 'beat': 46,
 'overpow': 47,
 'school': 48,
 'dark': 49,
 'demand': 50,
 'left': 51,
 'pleas': 52,
 'friend': 53,
 'sudden': 54,
 'trainer': 55,
 'battl': 56,
 'matur': 57,
 'hurtl': 58,
 'caus': 59,
 'rout': 60,
 'wield': 61,
 'faint': 62,
 'veng': 63,
 'miyamizu': 64,
 'god': 65,
 'undesir': 66,
 'crush': 67,
 'inosuk': 68,
 'e

In [26]:
index

{'0': [36],
 '1': [15],
 '2': [8],
 '3': [1, 33],
 '4': [30],
 '5': [2, 33],
 '6': [3],
 '7': [15],
 '8': [36, 39],
 '9': [4, 30, 39],
 '10': [25],
 '11': [34, 35],
 '12': [7, 28, 41, 42],
 '13': [15],
 '14': [28],
 '15': [40],
 '16': [26],
 '17': [47],
 '18': [48],
 '19': [19],
 '20': [4, 5, 14, 16, 17, 21, 23, 27, 34, 37, 39, 43, 44, 46],
 '21': [49],
 '22': [29],
 '23': [40],
 '24': [4, 11, 14, 18, 21],
 '25': [19, 20, 33],
 '26': [32, 49],
 '27': [1, 2, 46],
 '28': [1, 27],
 '29': [12],
 '30': [12],
 '31': [45],
 '32': [24, 28],
 '33': [1, 38],
 '34': [40],
 '35': [39],
 '36': [8],
 '37': [2, 6, 15],
 '38': [8, 45],
 '39': [6],
 '40': [12],
 '41': [14, 17],
 '42': [23],
 '43': [36],
 '44': [11, 13, 16, 20, 27, 29, 30, 36, 38, 40],
 '45': [2],
 '46': [13],
 '47': [27],
 '48': [12, 13, 14, 18, 20, 21, 26, 40, 50],
 '49': [16, 19, 31, 33, 47],
 '50': [17],
 '51': [1, 13, 25, 48],
 '52': [30],
 '53': [2, 4, 5, 7, 8, 9, 16, 20, 23, 31, 32, 36, 47],
 '54': [39],
 '55': [48],
 '56': [3, 8

In [27]:
rows = execute_query(vocabulary, index)
rows

robot


,animeTitle,animeDescription,Url
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n
45,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,https://myanimelist.net/anime/36862/Made_in_Ab...


### 2.2) Conjunctive query & Ranking score

### 2.2.1) Inverted index

Calculate $tfIdf$ as:
$$
\frac{tf_{i,j}}{{|n_{i,j}|}}
$$
where $tf_{i,j}$ is the number of time the word i is present in document j and ${|n_{i,j}|}$ is the lenght of the document (number of total words)

In [ ]:
original_df = df_from_3_information_file(path_documents_file)
cleaned_df = clean_df(original_df)

inverted_index = {}
for n_row in range(cleaned_df['animeDescription'].shape[0]):
    
    # Pick number of words in a single description
    description_row = cleaned_df['animeDescription'][n_row]
    n_words = len(description_row.split(" "))
    
    for (k,v) in vocabulary.items():
        if v not in inverted_index.keys():
            inverted_index[v] = []
        if k in cleaned_df['animeDescription'][n_row]:
            # n_row+1 becouse n_anime is n_row_in_dataset + 1
            term_frequency = k if description_row
            tfIdf = term_frequency/n_words
            inverted_index[v].append((n_row+1, tfIdf))
inverted_index
original_df

In [ ]:
original_df = df_from_3_information_file(path_documents_file)
cleaned_df = clean_df(original_df)

inverted_index = {}
for n_row in range(cleaned_df['animeDescription'].shape[0]):
    
    # Pick number of words in a single description
    description_row = cleaned_df['animeDescription'][n_row]
    n_words = len(description_row.split(" "))
    
    for (k,v) in vocabulary.items():
        if v not in inverted_index.keys():
            inverted_index[v] = []
        if k in cleaned_df['animeDescription'][n_row]:
            # n_row+1 becouse n_anime is n_row_in_dataset + 1
            inverted_index[v].append(n_row+1)
inverted_index

#### TF

In [55]:
## TF
# cacola term frequency per ogni documento, ma la chiave è la parola, non l'id
tf = []
for num_row in range(len(df)):
    doc = df.animeDescription.loc[num_row].split()
    len_doc = len(doc)
    counter = Counter(doc)
    for item, count in counter.items():
        counter[item] /= len_doc
    tf.append(counter)
tf

[Counter({'horrif': 0.009009009009009009,
          'alchemi': 0.009009009009009009,
          'experi': 0.009009009009009009,
          'goe': 0.009009009009009009,
          'wrong': 0.009009009009009009,
          'elric': 0.018018018018018018,
          'household': 0.009009009009009009,
          'brother': 0.02702702702702703,
          'edward': 0.036036036036036036,
          'alphons': 0.02702702702702703,
          'left': 0.009009009009009009,
          'catastroph': 0.009009009009009009,
          'new': 0.009009009009009009,
          'realiti': 0.009009009009009009,
          'ignor': 0.009009009009009009,
          'alchem': 0.009009009009009009,
          'principl': 0.009009009009009009,
          'ban': 0.009009009009009009,
          'human': 0.009009009009009009,
          'transmut': 0.009009009009009009,
          'boy': 0.018018018018018018,
          'attempt': 0.009009009009009009,
          'bring': 0.009009009009009009,
          'recent': 0.00900900900900900

In [ ]:
## IDF
# numdocs_cont_j = {}
# num_docs = len(df)
# for term_id in index:
#     numdocs_cont_j[term_id] = len(index[term_id])
#     idf = log(num_docs/numdocs_cont_j))
# numdocs_cont_j

####  IDF

In [82]:
## IDF
idf = {}
import numpy as np
numdocs_cont_j = {}
num_docs = len(df)
for term_id in index:
    numdocs_cont_j = len(index[term_id])
    idfscore = np.log(num_docs/numdocs_cont_j)
    idf[term_id] = idfscore
idf

{'0': 3.912023005428146,
 '1': 3.912023005428146,
 '2': 3.912023005428146,
 '3': 3.2188758248682006,
 '4': 3.912023005428146,
 '5': 3.2188758248682006,
 '6': 3.912023005428146,
 '7': 3.912023005428146,
 '8': 3.2188758248682006,
 '9': 2.8134107167600364,
 '10': 3.912023005428146,
 '11': 3.2188758248682006,
 '12': 2.5257286443082556,
 '13': 3.912023005428146,
 '14': 3.912023005428146,
 '15': 3.912023005428146,
 '16': 3.912023005428146,
 '17': 3.912023005428146,
 '18': 3.912023005428146,
 '19': 3.912023005428146,
 '20': 1.2729656758128876,
 '21': 3.912023005428146,
 '22': 3.912023005428146,
 '23': 3.912023005428146,
 '24': 2.302585092994046,
 '25': 2.8134107167600364,
 '26': 3.2188758248682006,
 '27': 2.8134107167600364,
 '28': 3.2188758248682006,
 '29': 3.912023005428146,
 '30': 3.912023005428146,
 '31': 3.912023005428146,
 '32': 3.2188758248682006,
 '33': 3.2188758248682006,
 '34': 3.912023005428146,
 '35': 3.912023005428146,
 '36': 3.912023005428146,
 '37': 2.8134107167600364,
 '38': 3

In [112]:
## per mappare le chiavi di tf in numeri(id)
tf_new = []
for d in tf:
    dic = dict((vocabulary[key], value) for (key, value) in d.items())
    tf_new.append(dic)
tf_new

[{682: 0.009009009009009009,
  640: 0.009009009009009009,
  949: 0.009009009009009009,
  632: 0.009009009009009009,
  1154: 0.009009009009009009,
  856: 0.018018018018018018,
  1330: 0.009009009009009009,
  1258: 0.02702702702702703,
  686: 0.036036036036036036,
  1485: 0.02702702702702703,
  51: 0.009009009009009009,
  1631: 0.009009009009009009,
  905: 0.009009009009009009,
  27: 0.009009009009009009,
  1279: 0.009009009009009009,
  1397: 0.009009009009009009,
  1857: 0.009009009009009009,
  1346: 0.009009009009009009,
  817: 0.009009009009009009,
  1696: 0.009009009009009009,
  1461: 0.018018018018018018,
  1105: 0.009009009009009009,
  238: 0.009009009009009009,
  1133: 0.009009009009009009,
  1314: 0.009009009009009009,
  170: 0.009009009009009009,
  1425: 0.009009009009009009,
  1859: 0.009009009009009009,
  1703: 0.009009009009009009,
  1094: 0.009009009009009009,
  1570: 0.009009009009009009,
  126: 0.009009009009009009,
  1027: 0.009009009009009009,
  1507: 0.01801801801801801

#### TF-IDF

In [160]:
term_id = {}
for term in idf: #for every term_id
    for i in range(len(tf_new)): #for every doc
        if int(term) in tf_new[i].keys(): #if term_id is the key of the document_i
            tupla = [i, idf[term]*tf_new[i][int(term)]]
            
            if term in term_id:
                term_id[term].append(tupla)
            else:
                term_id[term] = tupla
term_id

# non riesco ad aggiungerla come tupla(no append)

{'0': [35, 0.0395153838932136],
 '1': [14, 0.05015414109523264],
 '2': [7, 0.043955314667731976],
 '3': [0, 0.028998881305118924, [32, 0.028998881305118924]],
 '4': [29, 0.05216030673904195],
 '5': [1, 0.027990224564071308, [32, 0.028998881305118924]],
 '6': [2, 0.04890028756785183],
 '7': [14, 0.05015414109523264],
 '8': [35, 0.03251389722089092, [38, 0.03095072908527116]],
 '9': [3,
  0.02534604249333366,
  [29, 0.03751214289013382],
  [38, 0.027052026122692658]],
 '10': [24, 0.04206476349922738],
 '11': [33, 0.056971253537490274, [34, 0.11495985088815001]],
 '12': [6,
  0.02500721430008174,
  [27, 0.01884872122618101],
  [40, 0.0531732346170159],
  [41, 0.03459902252477062]],
 '13': [14, 0.05015414109523264],
 '14': [27, 0.029194201533045864],
 '15': [39, 0.14312279288151752],
 '16': [25, 0.050805493576988915],
 '17': [46, 0.12225071891962956],
 '18': [47, 0.06112535945981478],
 '19': [18, 0.050805493576988915],
 '20': [3,
  0.02293631848311509,
  [4, 0.014465519043328268],
  [13, 0

In [ ]:
# term_id = {}
# for term in idf: #for every term_id
#     for i in range(len(tf_new)): #for every doc
#         if int(term) in tf_new[i].keys(): #if term_id is the key of the document_i

#             tupla = (i, idf[term]*tf_new[i][int(term)])
#     term_id[term] = tupla
# term_id

In [ ]:
# term_id = {}
# for term in idf: #for every term_id
#     for doc in tf_new: #for every doc
#         if int(term) in doc.keys(): #if term_id is the key of the document_i
#             tupla = (doc, idf[term]*doc[int(term)])
#     term_id[term] = tupla
# term_id